In [2]:
import cv2
import numpy as np
import os
import random

############################
#                          #
#  Folder Setup            #
#                          #
############################
input_folder = "LIS-fingerspelling-dataset/LIS-fingerspelling-dataset"  
output_folder = "LIS-fingerspelling-dataset/LIS-fingerspelling-dataset-processed"
os.makedirs(output_folder, exist_ok=True)

valid_exts = (".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG")

############################
#                          #
#  Background Colors       #
#                          #
############################

fixed_bg_colors = [
    (255, 255, 255),  # white
    (0, 0, 0),        # black
    (0, 255, 0),      # green
    (255, 0, 0),      # blue
    (0, 255, 255)     # yellow
]
num_random_colors = 3


############################
#                          #
#  Image Searching         #
#                          #
############################
def find_images_recursively(root_path, exts=valid_exts):

    image_paths = []
    for dirpath, dirnames, filenames in os.walk(root_path):
        for filename in filenames:
            if filename.lower().endswith(exts):
                full_path = os.path.join(dirpath, filename)
                image_paths.append(full_path)
    return image_paths


###############################
#                             #
#  Finding Hand with Contour  #
#                             #
###############################
def isolate_hand_by_contour(img_bgr):

    #converting BGR to grayscale
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    #Otsu's threshold automatically picks a threshold
    _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    #morphological operations to remove noise
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=1)

    #finding external contours
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        
        img_bgra = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2BGRA)
        img_bgra[..., 3] = 0
        return img_bgra

    #picking the largest contour by area
    largest_contour = max(contours, key=cv2.contourArea)

    #creating a new mask with just that contour
    contour_mask = np.zeros_like(mask)
    cv2.drawContours(contour_mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

    #converting original to BGRA
    img_bgra = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2BGRA)

    img_bgra[contour_mask == 0, 3] = 0

    return img_bgra

###############################
#                             #
#  Add Hand with Background   #
#                             #
###############################
def composite_on_color(fg_bgra, bg_color):

    h, w = fg_bgra.shape[:2]

    #creating solid BGRA background
    bg_bgra = np.zeros((h, w, 4), dtype=np.uint8)
    bg_bgra[..., 0] = bg_color[0]  # B
    bg_bgra[..., 1] = bg_color[1]  # G
    bg_bgra[..., 2] = bg_color[2]  # R
    bg_bgra[..., 3] = 255          # alpha=255

    #extracting alpha from the foreground
    alpha_fg = fg_bgra[..., 3].astype(float) / 255.0
    alpha_bg = bg_bgra[..., 3].astype(float) / 255.0

    out_bgra = np.zeros_like(fg_bgra, dtype=np.uint8)

    for c in range(3):  # B, G, R
        out_bgra[..., c] = (
            fg_bgra[..., c] * alpha_fg + 
            bg_bgra[..., c] * (1.0 - alpha_fg)
        ).astype(np.uint8)

    #making final alpha fully opaque
    out_bgra[..., 3] = 255

    #return only background
    return out_bgra[..., :3]

if __name__ == "__main__":
    
    image_paths = find_images_recursively(input_folder)
    print(f"Found {len(image_paths)} images in '{input_folder}'.")

    for img_path in image_paths:
        rel_path = os.path.relpath(img_path, input_folder)  # e.g. "A/img1.jpg"
        base_name, ext = os.path.splitext(rel_path)

        img_bgr = cv2.imread(img_path)
        if img_bgr is None:
            print(f"Could not load {img_path}, skipping.")
            continue

        ###############################
        #                             #
        #  Finding hand contour       #
        #                             #
        ###############################
        hand_bgra = isolate_hand_by_contour(img_bgr)

        ##################################
        #                                #
        #  putting hand with backgrounds #
        #                                #
        ##################################
        backgrounds = list(fixed_bg_colors)
        
        for _ in range(num_random_colors):
            random_bg = (
                random.randint(0,255),
                random.randint(0,255),
                random.randint(0,255)
            )
            backgrounds.append(random_bg)

        subfolder = os.path.dirname(base_name)  
        filename_no_ext = os.path.basename(base_name)
        out_subfolder = os.path.join(output_folder, subfolder)
        os.makedirs(out_subfolder, exist_ok=True)

        for bg_col in backgrounds:
            out_bgr = composite_on_color(hand_bgra, bg_col)

            color_name = f"BG_{bg_col[2]}_{bg_col[1]}_{bg_col[0]}"
            out_filename = f"{filename_no_ext}_{color_name}.png"
            out_fullpath = os.path.join(out_subfolder, out_filename)

            cv2.imwrite(out_fullpath, out_bgr)

        print(f"Processed: {rel_path}, created {len(backgrounds)} variants.")

Found 6428 images in 'LIS-fingerspelling-dataset/LIS-fingerspelling-dataset'.
Processed: a\a_ANTONIA_10_front.JPG, created 8 variants.
Processed: a\a_ANTONIA_11_front.JPG, created 8 variants.
Processed: a\a_ANTONIA_12_front.JPG, created 8 variants.
Processed: a\a_ANTONIA_19_top.JPG, created 8 variants.
Processed: a\a_ANTONIA_1_bottom.JPG, created 8 variants.
Processed: a\a_ANTONIA_20_top.JPG, created 8 variants.
Processed: a\a_ANTONIA_21_top.JPG, created 8 variants.
Processed: a\a_ANTONIA_22_top.JPG, created 8 variants.
Processed: a\a_ANTONIA_23_top.JPG, created 8 variants.
Processed: a\a_ANTONIA_24_top.JPG, created 8 variants.
Processed: a\a_ANTONIA_25_top.JPG, created 8 variants.
Processed: a\a_ANTONIA_2_bottom.JPG, created 8 variants.
Processed: a\a_ANTONIA_3_bottom.JPG, created 8 variants.
Processed: a\a_ANTONIA_4_bottom.JPG, created 8 variants.
Processed: a\a_ANTONIA_5_bottom.JPG, created 8 variants.
Processed: a\a_ANTONIA_6_bottom.JPG, created 8 variants.
Processed: a\a_ANTONIA_7

Processed: a\a_IBRAHIM_6_bottom.JPG, created 8 variants.
Processed: a\a_IBRAHIM_7_bottom.JPG, created 8 variants.
Processed: a\a_IBRAHIM_8_bottom.JPG, created 8 variants.
Processed: a\a_IBRAHIM_9_front.JPG, created 8 variants.
Processed: a\a_KENZA_10_front.JPG, created 8 variants.
Processed: a\a_KENZA_11_front.JPG, created 8 variants.
Processed: a\a_KENZA_12_front.JPG, created 8 variants.
Processed: a\a_KENZA_13_front.JPG, created 8 variants.
Processed: a\a_KENZA_14_front.JPG, created 8 variants.
Processed: a\a_KENZA_1_bottom.JPG, created 8 variants.
Processed: a\a_KENZA_28_top.JPG, created 8 variants.
Processed: a\a_KENZA_29_top.JPG, created 8 variants.
Processed: a\a_KENZA_2_bottom.JPG, created 8 variants.
Processed: a\a_KENZA_30_top.JPG, created 8 variants.
Processed: a\a_KENZA_31_top.JPG, created 8 variants.
Processed: a\a_KENZA_32_top.JPG, created 8 variants.
Processed: a\a_KENZA_33_top.JPG, created 8 variants.
Processed: a\a_KENZA_3_bottom.JPG, created 8 variants.
Processed: a\a_

Processed: b\b_FATIMA_18_front.JPG, created 8 variants.
Processed: b\b_FATIMA_19_front.JPG, created 8 variants.
Processed: b\b_FATIMA_1_bottom.JPG, created 8 variants.
Processed: b\b_FATIMA_2_bottom.JPG, created 8 variants.
Processed: b\b_FATIMA_3_bottom.JPG, created 8 variants.
Processed: b\b_FATIMA_42_top.JPG, created 8 variants.
Processed: b\b_FATIMA_43_top.JPG, created 8 variants.
Processed: b\b_FATIMA_44_top.JPG, created 8 variants.
Processed: b\b_FATIMA_45_top.JPG, created 8 variants.
Processed: b\b_FATIMA_46_top.JPG, created 8 variants.
Processed: b\b_FATIMA_47_top.JPG, created 8 variants.
Processed: b\b_FATIMA_48_top.JPG, created 8 variants.
Processed: b\b_FATIMA_49_top.JPG, created 8 variants.
Processed: b\b_FATIMA_4_bottom.JPG, created 8 variants.
Processed: b\b_FATIMA_50_top.JPG, created 8 variants.
Processed: b\b_FATIMA_51_top.JPG, created 8 variants.
Processed: b\b_FATIMA_52_top.JPG, created 8 variants.
Processed: b\b_FATIMA_5_bottom.JPG, created 8 variants.
Processed: b\b

Processed: b\b_MIKE_13_front.JPG, created 8 variants.
Processed: b\b_MIKE_14_front.JPG, created 8 variants.
Processed: b\b_MIKE_15_front.JPG, created 8 variants.
Processed: b\b_MIKE_1_bottom.JPG, created 8 variants.
Processed: b\b_MIKE_2_bottom.JPG, created 8 variants.
Processed: b\b_MIKE_33_top.JPG, created 8 variants.
Processed: b\b_MIKE_34_top.JPG, created 8 variants.
Processed: b\b_MIKE_35_top.JPG, created 8 variants.
Processed: b\b_MIKE_36_top.JPG, created 8 variants.
Processed: b\b_MIKE_37_top.JPG, created 8 variants.
Processed: b\b_MIKE_38_top.JPG, created 8 variants.
Processed: b\b_MIKE_39_top.JPG, created 8 variants.
Processed: b\b_MIKE_3_bottom.JPG, created 8 variants.
Processed: b\b_MIKE_40_top.JPG, created 8 variants.
Processed: b\b_MIKE_41_top.JPG, created 8 variants.
Processed: b\b_MIKE_42_top.JPG, created 8 variants.
Processed: b\b_MIKE_43_top.JPG, created 8 variants.
Processed: b\b_MIKE_44_top.JPG, created 8 variants.
Processed: b\b_MIKE_45_top.JPG, created 8 variants.


Processed: c\c_GIANLUCA_3_bottom.JPG, created 8 variants.
Processed: c\c_GIANLUCA_40_top.JPG, created 8 variants.
Processed: c\c_GIANLUCA_41_top.JPG, created 8 variants.
Processed: c\c_GIANLUCA_42_top.JPG, created 8 variants.
Processed: c\c_GIANLUCA_43_top.JPG, created 8 variants.
Processed: c\c_GIANLUCA_44_front.JPG, created 8 variants.
Processed: c\c_GIANLUCA_45_front.JPG, created 8 variants.
Processed: c\c_GIANLUCA_46_front.JPG, created 8 variants.
Processed: c\c_GIANLUCA_47_front.JPG, created 8 variants.
Processed: c\c_GIANLUCA_48_front.JPG, created 8 variants.
Processed: c\c_GIANLUCA_49_front.JPG, created 8 variants.
Processed: c\c_GIANLUCA_4_bottom.JPG, created 8 variants.
Processed: c\c_GIANLUCA_50_front.JPG, created 8 variants.
Processed: c\c_GIANLUCA_51_front.JPG, created 8 variants.
Processed: c\c_GIANLUCA_52_front.JPG, created 8 variants.
Processed: c\c_GIANLUCA_53_front.JPG, created 8 variants.
Processed: c\c_GIANLUCA_5_bottom.JPG, created 8 variants.
Processed: c\c_GIANLUC

...

Processed: w\w_NOOR_2_bottom.JPG, created 8 variants.
Processed: w\w_NOOR_30_top.JPG, created 8 variants.
Processed: w\w_NOOR_31_top.JPG, created 8 variants.
Processed: w\w_NOOR_3_bottom.JPG, created 8 variants.
Processed: w\w_NOOR_4_bottom.JPG, created 8 variants.
Processed: w\w_NOOR_5_bottom.JPG, created 8 variants.
Processed: w\w_NOOR_6_front.JPG, created 8 variants.
Processed: w\w_NOOR_7_front.JPG, created 8 variants.
Processed: w\w_NOOR_8_front.JPG, created 8 variants.
Processed: w\w_NOOR_9_front.JPG, created 8 variants.
Processed: x\x_ANTONIA_10_front.JPG, created 8 variants.
Processed: x\x_ANTONIA_11_front.JPG, created 8 variants.
Processed: x\x_ANTONIA_12_front.JPG, created 8 variants.
Processed: x\x_ANTONIA_13_front.JPG, created 8 variants.
Processed: x\x_ANTONIA_14_front.JPG, created 8 variants.
Processed: x\x_ANTONIA_15_front.JPG, created 8 variants.
Processed: x\x_ANTONIA_16_front.JPG, created 8 variants.
Processed: x\x_ANTONIA_1_bottom.JPG, created 8 variants.
Processed: x

Processed: x\x_GIULIA_39_front.JPG, created 8 variants.
Processed: x\x_GIULIA_3_bottom.JPG, created 8 variants.
Processed: x\x_GIULIA_4_bottom.JPG, created 8 variants.
Processed: x\x_GIULIA_5_bottom.JPG, created 8 variants.
Processed: x\x_GIULIA_6_bottom.JPG, created 8 variants.
Processed: x\x_GIULIA_7_bottom.JPG, created 8 variants.
Processed: x\x_GIULIA_8_bottom.JPG, created 8 variants.
Processed: x\x_HUZAIFA_10_front.JPG, created 8 variants.
Processed: x\x_HUZAIFA_11_front.JPG, created 8 variants.
Processed: x\x_HUZAIFA_12_front.JPG, created 8 variants.
Processed: x\x_HUZAIFA_13_front.JPG, created 8 variants.
Processed: x\x_HUZAIFA_1_bottom.JPG, created 8 variants.
Processed: x\x_HUZAIFA_23_top.JPG, created 8 variants.
Processed: x\x_HUZAIFA_24_top.JPG, created 8 variants.
Processed: x\x_HUZAIFA_25_top.JPG, created 8 variants.
Processed: x\x_HUZAIFA_26_top.JPG, created 8 variants.
Processed: x\x_HUZAIFA_27_top.JPG, created 8 variants.
Processed: x\x_HUZAIFA_2_bottom.JPG, created 8 v

Processed: x\x_MIKE_6_bottom.JPG, created 8 variants.
Processed: x\x_MIKE_7_bottom.JPG, created 8 variants.
Processed: x\x_MIKE_8_bottom.JPG, created 8 variants.
Processed: x\x_MIKE_9_bottom.JPG, created 8 variants.
Processed: x\x_NOOR_10_front.JPG, created 8 variants.
Processed: x\x_NOOR_11_front.JPG, created 8 variants.
Processed: x\x_NOOR_1_bottom.JPG, created 8 variants.
Processed: x\x_NOOR_2_bottom.JPG, created 8 variants.
Processed: x\x_NOOR_30_top.JPG, created 8 variants.
Processed: x\x_NOOR_31_top.JPG, created 8 variants.
Processed: x\x_NOOR_32_top.JPG, created 8 variants.
Processed: x\x_NOOR_33_top.JPG, created 8 variants.
Processed: x\x_NOOR_34_top.JPG, created 8 variants.
Processed: x\x_NOOR_35_top.JPG, created 8 variants.
Processed: x\x_NOOR_36_top.JPG, created 8 variants.
Processed: x\x_NOOR_37_top.JPG, created 8 variants.
Processed: x\x_NOOR_38_top.JPG, created 8 variants.
Processed: x\x_NOOR_39_top.JPG, created 8 variants.
Processed: x\x_NOOR_3_bottom.JPG, created 8 vari

Processed: y\y_GIANLUCA_38_top.JPG, created 8 variants.
Processed: y\y_GIANLUCA_39_top.JPG, created 8 variants.
Processed: y\y_GIANLUCA_3_bottom.JPG, created 8 variants.
Processed: y\y_GIANLUCA_40_front.JPG, created 8 variants.
Processed: y\y_GIANLUCA_41_front.JPG, created 8 variants.
Processed: y\y_GIANLUCA_42_front.JPG, created 8 variants.
Processed: y\y_GIANLUCA_43_front.JPG, created 8 variants.
Processed: y\y_GIANLUCA_44_front.JPG, created 8 variants.
Processed: y\y_GIANLUCA_45_front.JPG, created 8 variants.
Processed: y\y_GIANLUCA_46_front.JPG, created 8 variants.
Processed: y\y_GIANLUCA_47_front.JPG, created 8 variants.
Processed: y\y_GIANLUCA_4_bottom.JPG, created 8 variants.
Processed: y\y_GIANLUCA_5_bottom.JPG, created 8 variants.
Processed: y\y_GIANLUCA_6_bottom.JPG, created 8 variants.
Processed: y\y_GIANLUCA_7_bottom.JPG, created 8 variants.
Processed: y\y_GIANLUCA_8_bottom.JPG, created 8 variants.
Processed: y\y_GIANLUCA_9_bottom.JPG, created 8 variants.
Processed: y\y_GIU

Processed: y\y_MIKE_21_front.JPG, created 8 variants.
Processed: y\y_MIKE_22_front.JPG, created 8 variants.
Processed: y\y_MIKE_2_bottom.JPG, created 8 variants.
Processed: y\y_MIKE_35_top.JPG, created 8 variants.
Processed: y\y_MIKE_36_top.JPG, created 8 variants.
Processed: y\y_MIKE_37_top.JPG, created 8 variants.
Processed: y\y_MIKE_38_top.JPG, created 8 variants.
Processed: y\y_MIKE_39_top.JPG, created 8 variants.
Processed: y\y_MIKE_3_bottom.JPG, created 8 variants.
Processed: y\y_MIKE_40_top.JPG, created 8 variants.
Processed: y\y_MIKE_41_top.JPG, created 8 variants.
Processed: y\y_MIKE_42_top.JPG, created 8 variants.
Processed: y\y_MIKE_43_top.JPG, created 8 variants.
Processed: y\y_MIKE_44_top.JPG, created 8 variants.
Processed: y\y_MIKE_45_top.JPG, created 8 variants.
Processed: y\y_MIKE_4_bottom.JPG, created 8 variants.
Processed: y\y_MIKE_5_bottom.JPG, created 8 variants.
Processed: y\y_MIKE_6_bottom.JPG, created 8 variants.
Processed: y\y_MIKE_7_bottom.JPG, created 8 varian